In [2]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
import numpy as np
import sys, os, string, random

characters = string.printable
char_indices = dict((c, i) for i, c in enumerate(characters))
indices_char = dict((i, c) for i, c in enumerate(characters))

INPUT_VOCAB_SIZE = len(characters)
BATCH_SIZE = 200
HIDDEN_SIZE = 100
TIME_STEPS = 3

def encode_one_hot(line):
    x = np.zeros((len(line), INPUT_VOCAB_SIZE))
    for i, c in enumerate(line):
        index = char_indices[c] if c in characters else char_indices[' ']
        x[i][index] = 1 
    return x

def decode_one_hot(x):
    s = []
    for onehot in x:
        one_index = np.argmax(onehot) 
        s.append(indices_char[one_index]) 
    return ''.join(s)
    
def prepare_for_rnn(x):
    # All slices of size TIME_STEPS, sliding through x
    ind = [np.array(np.arange(i, i+TIME_STEPS)) for i in range(x.shape[0] - TIME_STEPS + 1)]
    ind = np.array(ind, dtype=np.int32)
    x_rnn = x[ind]
    return x_rnn

def input_generator(nsamples):
    def generate_line():
        inline = [' ']; outline = []
        for _ in range(nsamples):
            c = random.choice(characters) 
            expected = c.lower() if c in string.ascii_letters else ' ' 
            inline.append(c); outline.append(expected)
        inline.append(' '); 
        for i in range(nsamples):
            if outline[i] == ' ': continue
            if i > 0 and i < nsamples-1:
                if outline[i-1] == ' ' and outline[i+1] == ' ':
                    outline[i] = ' '
            if (i == 0 and outline[1] == ' ') or (i == nsamples-1 and outline[nsamples-2] == ' '):
                outline[i] = ' '
        return ''.join(inline), ''.join(outline)

    while True:
        input_data, expected = generate_line()
        data_in = encode_one_hot(input_data)
        data_out = encode_one_hot(expected)
        yield prepare_for_rnn(data_in), data_out

def train(model):
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    input_gen = input_generator(BATCH_SIZE)
    validation_gen = input_generator(BATCH_SIZE)
    model.fit_generator(input_gen,
                epochs = 50, workers=1,
                steps_per_epoch = 50,
                validation_data = validation_gen,
                validation_steps = 10)

def build_model():
    model = Sequential()
    model.add(SimpleRNN(HIDDEN_SIZE, input_shape=(None, INPUT_VOCAB_SIZE)))
    model.add(Dense(INPUT_VOCAB_SIZE, activation='softmax'))
    return model

model = build_model()
model.summary()
train(model)

input("Network has been trained. Press <Enter> to run program.")
fp = "../pride-and-prejudice.txt"
# sys.argv[1]
with open(fp) as f:
    for line in f:
        if line.isspace(): continue
        batch = prepare_for_rnn(encode_one_hot(" " + line + " "))
        #batch = prepare_for_rnn(encode_one_hot(line))
        preds = model.predict(batch)
        normal = decode_one_hot(preds)
        print(normal)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 100)               20100     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
Total params: 30,200
Trainable params: 30,200
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50


/var/folders/lx/pk8l0jrj46s2mmm8c8m38p1m0000gn/T/ipykernel_29058/911577950.py:65: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(input_gen,


 4/50 [=>............................] - ETA: 0s - loss: 4.5936 - accuracy: 0.0175 

2022-06-02 15:16:10.768914: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 2s 21ms/step - loss: 3.1027 - accuracy: 0.4670 - val_loss: 1.8722 - val_accuracy: 0.5950
Epoch 2/50
 1/50 [..............................] - ETA: 0s - loss: 1.7251 - accuracy: 0.6450

...

3/3 [==============================] - 0s 8ms/step
staid so long  that even bingley   good humour was overcome  and he 
2/2 [==============================] - 0s 5ms/step
proceeded so far as to talk of giving them   hint to be gone  
3/3 [==============================] - 0s 5ms/step
miss bingley was very deeply mortified by darcy   marriage  but as she 
3/3 [==============================] - 0s 5ms/step
thought it advisable to retain the right of visiting at pemberley  she 
3/3 [==============================] - 0s 5ms/step
dropt all her resentment  was fonder than ever of georgiana  almost as 
3/3 [==============================] - 0s 5ms/step
attentive to darcy as heretofore  and paid off every arrear of civility 
1/1 [==============================] - 0s 13ms/step
to elizabeth  
3/3 [==============================] - 0s 5ms/step
pemberley was now georgiana   home  and the attachment of the sisters 
3/3 [==============================] - 0s 5ms/step
was exactly what darcy had hoped 

3/3 [==============================] - 0s 8ms/step
 trademark copyright  agreement   if you do not agree to abide by all 
3/3 [==============================] - 0s 8ms/step
the terms of this agreement  you must cease using and return or destroy 
3/3 [==============================] - 0s 5ms/step
all copies of project gutenberg tm electronic works in your possession  
3/3 [==============================] - 0s 7ms/step
if you paid   fee for obtaining   copy of or access to   project 
3/3 [==============================] - 0s 6ms/step
gutenberg tm electronic work and you do not agree to be bound by the 
3/3 [==============================] - 0s 8ms/step
terms of this agreement  you may obtain   refund from the person or 
3/3 [==============================] - 0s 5ms/step
entity to whom you paid the fee as set forth in paragraph        
3/3 [==============================] - 0s 5ms/step
       project gutenberg  is   registered trademark   it may only be 
3/3 [=============================

2/2 [==============================] - 0s 12ms/step
work or any other work associated with project gutenberg tm  
3/3 [==============================] - 0s 5ms/step
        do not copy  display  perform  distribute or redistribute this 
2/2 [==============================] - 0s 12ms/step
electronic work  or any part of this electronic work  without 
3/3 [==============================] - 0s 5ms/step
prominently displaying the sentence set forth in paragraph       with 
3/3 [==============================] - 0s 6ms/step
active links or immediate access to the full terms of the project 
1/1 [==============================] - 0s 19ms/step
gutenberg tm license  
3/3 [==============================] - 0s 5ms/step
        you may convert to and distribute this work in any binary  
3/3 [==============================] - 0s 5ms/step
compressed  marked up  nonproprietary or proprietary form  including any 
3/3 [==============================] - 0s 5ms/step
word processing or hypertext form   ho

3/3 [==============================] - 0s 5ms/step
liability  breach of warranty or breach of contract except those 
2/2 [==============================] - 0s 12ms/step
provided in paragraph      you agree that the foundation  the 
3/3 [==============================] - 0s 5ms/step
trademark owner  and any distributor under this agreement will not be 
3/3 [==============================] - 0s 5ms/step
liable to you for actual  direct  indirect  consequential  punitive or 
3/3 [==============================] - 0s 5ms/step
incidental damages even if you give notice of the possibility of such 
1/1 [==============================] - 0s 14ms/step
damage  
3/3 [==============================] - 0s 5ms/step
        limited right of replacement or refund   if you discover   
3/3 [==============================] - 0s 5ms/step
defect in this electronic work within    days of receiving it  you can 
3/3 [==============================] - 0s 5ms/step
receive   refund of the money  if any  you paid

1/1 [==============================] - 0s 13ms/step
page at http   pglaf org 
2/2 [==============================] - 0s 11ms/step
for additional contact information  
1/1 [==============================] - 0s 14ms/step
     dr  gregory    newby 
2/2 [==============================] - 0s 10ms/step
     chief executive and director 
1/1 [==============================] - 0s 20ms/step
     gbnewby pglaf org 
3/3 [==============================] - 0s 5ms/step
section     information about donations to the project gutenberg 
1/1 [==============================] - 0s 19ms/step
literary archive foundation 
3/3 [==============================] - 0s 5ms/step
project gutenberg tm depends upon and cannot survive without wide 
2/2 [==============================] - 0s 5ms/step
spread public support and donations to carry out its mission of 
3/3 [==============================] - 0s 6ms/step
increasing the number of public domain and licensed works that can be 
3/3 [==============================] 